In [2]:
import pandas as pd

In [3]:
car_kick = pd.read_csv("data/car_kick.csv")

In [5]:
car_kick.head()

,PurchDate,VehYear,VehicleAge,VehOdo,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,...,WheelTypeID,WheelType,Nationality,Size,TopThreeAmericanName,BYRNO,VNZIP1,VNST,IsOnlineSale,Class
0,1.289952e+09,2006.0,4,51954.0,6197.0,7062.0,9605.0,10426.0,5341.0,6351.0,...,2.0,Covers,AMERICAN,MEDIUM,CHRYSLER,99750,32124,FL,0,0
1,1.242691e+09,2005.0,4,89127.0,3688.0,4783.0,4483.0,5666.0,3688.0,4783.0,...,2.0,Covers,AMERICAN,MEDIUM,FORD,20833,78754,TX,0,0
2,1.248221e+09,2006.0,3,71271.0,6897.0,8449.0,7949.0,9625.0,6868.0,8549.0,...,1.0,Alloy,AMERICAN,LARGE,GM,22916,80011,CO,0,0
3,1.285718e+09,2008.0,2,83338.0,7878.0,8925.0,11723.0,13026.0,7801.0,8704.0,...,2.0,Covers,AMERICAN,LARGE,GM,23657,94544,CA,0,0
4,1.237334e+09,2007.0,2,58698.0,8800.0,10091.0,10004.0,11398.0,7355.0,8543.0,...,1.0,Alloy,AMERICAN,LARGE,GM,20833,77086,TX,0,0
